In [2]:
import sqlite3

db = sqlite3.connect('IMDB.sqlite')
db

In [4]:
db = sqlite3.connect('School.sqlite')
db

In [1]:
import sqlite3

In [19]:
import sqlite3
import time

# Specify the path to your mbox.txt file
filename = 'mbox.txt'  # Update this if it's in a different location

# Create a new SQLite database or connect to an existing one
conn = sqlite3.connect('emaildb.sqlite', timeout=10)  # This will create 'emaildb.sqlite' in your current directory
cur = conn.cursor()

# Create the Counts table
cur.execute('''
CREATE TABLE IF NOT EXISTS Counts (
    org TEXT,
    count INTEGER
)
''')

# Function to execute a command with retry
def execute_with_retry(query, params=None, retries=5):
    for attempt in range(retries):
        try:
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            return  # Exit the function if successful
        except sqlite3.OperationalError as e:
            if "locked" in str(e):
                print("Database is locked, retrying...")
                time.sleep(1)  # Wait before retrying
            else:
                raise  # Raise the error if it's not a locking issue

# Clear the table before inserting new data
execute_with_retry('DELETE FROM Counts')

# Open the mbox file and process it
with open(filename, 'r') as fh:
    for line in fh:
        if not line.startswith('From '):
            continue
        words = line.split()
        email = words[1]
        domain = email.split('@')[1]

        # Insert or update the count in the database
        execute_with_retry('SELECT count FROM Counts WHERE org = ?', (domain,))
        row = cur.fetchone()
        if row is None:
            execute_with_retry('INSERT INTO Counts (org, count) VALUES (?, ?)', (domain, 1))
        else:
            execute_with_retry('UPDATE Counts SET count = count + 1 WHERE org = ?', (domain,))

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()

print("Database 'emaildb.sqlite' has been created with email counts.")


DatabaseError: file is not a database

In [20]:
cur.execute('del emaildb.sqlite')

OperationalError: near "del": syntax error